In [13]:
import requests, json
import urllib.parse
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import requests
from io import BytesIO
import torch
torch.device("cuda", 1)
# Load the OpenAI CLIP Model
print('Loading CLIP Model...')
model = SentenceTransformer('clip-ViT-B-32')

Loading CLIP Model...


In [10]:
url = "http://localhost:5000/search-text"

data = {
    "text_query": urllib.parse.quote_plus("Julian Castro at his announcement in San Antonio, Tex., on Saturday, Jan. 12, 2019."),
}

headers = {'Content-type': 'application/json'}
r = requests.post(url, headers=headers, data=json.dumps(data))
#r.json to get the response as json
result = r.json()

In [6]:
result['links']

['https://dallasnews.imgix.net/1546486600-Castro-event-announcement.png',
 'https://thumbnails.texastribune.org/_luaeP_H9k1hKZeE4v7H7dmpx20=/850x570/smart/filters:quality(75)/https://static.texastribune.org/media/files/73646110321f4dcb10385a5d75b9293f/RCJ_7575.jpg',
 'https://i.guim.co.uk/img/media/bd0829730314ac65451bdfe2fdc9b4c1c0383580/0_260_3900_2340/master/3900.jpg?width=465&quality=45&auto=format&fit=max&dpr=2&s=cc8008e02412aebda8d65230396f093c',
 'https://static01.nyt.com/images/2019/01/13/us/13CASTRO-p1/13CASTRO-p1-videoSixteenByNine3000-v2.jpg?year=2019&h=1688&w=3000&s=8a78d985fff20740d1c4de551ba9e0cd417f5c53e6b59dc2961820256d5476c4&k=ZQJBKqZ0VN&tw=1',
 'http://thumbor-prod-us-east-1.photo.aws.arc.pub/l97umGzmM3NRK4krnelLaewdOLQ=/arc-anglerfish-arc2-prod-dmn/public/VO5ZDYA5XDKL3NHZR5ZF2GDUZ4.jpg',
 'https://thumbnails.texastribune.org/zkWihZckBkcfAovCUCOi_Mz4b4s=/850x570/smart/filters:quality(75)/https://static.texastribune.org/media/files/325c3b308907ebeb7436381f61e4b907/Juli

In [14]:

# Next we compute the embeddings
# To encode an image, you can use the following code:
# from PIL import Image
# encoded_image = model.encode(Image.open(filepath))
embeddings1 = model.encode(Image.open('/root/thesis/dataset_visualizer/static/test/0.jpg'),batch_size=128, convert_to_tensor=True)
# response = requests.get(result['links'][3])
# img_list.append(Image.open(BytesIO(response.content)))
cosine_scores = []
for url in result['links'][:5]:
    response = requests.get(url)
    embeddings2 = model.encode(Image.open(BytesIO(response.content)),batch_size=128, convert_to_tensor=True)
    cosine_scores.append(util.cos_sim(embeddings1, embeddings2)) 

# encoded_image = model.encode([Image.open(filepath) for filepath in image_names], batch_size=128, convert_to_tensor=True, show_progress_bar=True)
# encoded_image = model.encode(img_list, batch_size=128, convert_to_tensor=True, show_progress_bar=True)
#Compute embedding for both lists
# embeddings1 = model.encode(img_list[0],batch_size=128, convert_to_tensor=True)
# embeddings2 = model.encode(img_list[1],batch_size=128, convert_to_tensor=True)
# Now we run the clustering algorithm. This function compares images aganist 
# all other images and returns a list with the pairs that have the highest 
# cosine similarity score
# processed_images = util.paraphrase_mining_embeddings(encoded_image)
# NUM_SIMILAR_IMAGES = 10 
# cosine_scores = util.cos_sim(embeddings1, embeddings2)

# =================
# NEAR DUPLICATES
# =================
print('Finding near duplicate images...')
# Use a threshold parameter to identify two images as similar. By setting the threshold lower, 
# you will get larger clusters which have less similar images in it. Threshold 0 - 1.00
# A threshold of 1.00 means the two images are exactly the same. Since we are finding near 
# duplicate images, we can set it at 0.99 or any number 0 < X < 1.00.
for score, image_id1, image_id2 in cosine_scores:
    print("\nScore: {:.3f}%".format(score * 100))
    if image_id1 == 0:
        print('0')
    else:
        print(result['links'][image_id1-1])
    if image_id2 == 0:
        print('0')
    else:
        print(result['links'][image_id2-1])

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.